In [18]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array


result = pd.read_csv('preprocessed_processed_image.csv')
print(result)


        선종  상태  state  vessel_type  nation_type
0     낚시어선  조업      0            8            1
1      범장망  조업      0            3            0
2     낚시어선  표류      2            8            1
3     등광조망  이동      1            2            0
4     등광조망  표류      2            2            0
...    ...  ..    ...          ...          ...
1576   안강망  조업      0            4            1
1577    연승  표류      2            5            1
1578  등광조망  조업      0            2            0
1579  등광조망  조업      0            2            0
1580  등광조망  이동      1            2            0

[1581 rows x 5 columns]


In [8]:
# 모델 불러오기
from tensorflow.keras.models import load_model
model = load_model('mobilenet_model.h5')

In [28]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

# 이미지 디렉토리 경로
image_dir = 'image'  # 원본 이미지 경로
# output_dir = 'processed_images'  # 저장할 폴더 경로

# 저장할 폴더가 없으면 생성
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# 이미지 읽기 및 전처리
images = []
filenames = []  # 파일 이름을 저장할 리스트
for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png')):  # 여러 확장자를 처리
        img_path = os.path.join(image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 정규화
            images.append(image_array)
            # filenames.append(filename)  # 파일 이름을 저장하여 나중에 이미지 순서 매칭
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
images = np.array(images)

# 이미지 파일을 새로운 폴더에 순서대로 저장 (번호를 매겨 저장)
# for idx, (image_array, filename) in enumerate(zip(images, filenames)):
#     # 이미지 배열을 다시 PIL 이미지로 변환
#     image = array_to_img(image_array)

#     # 저장할 경로 설정 (번호를 붙인 파일명으로 저장)
#     save_filename = f"{idx+1:03d}_{filename}"  # 예: 001_filename.jpg
#     save_path = os.path.join(output_dir, save_filename)

#     # 이미지 저장
#     image.save(save_path)
#     print(f"Image {filename} saved as {save_filename} at {save_path}")


# 레이블을 정의 (예: 0 또는 1)
# 레이블을 적절히 설정하세요
labels = data['vessel_type']
labels = to_categorical(labels, num_classes=10)



In [29]:
target = result['vessel_type']
target = to_categorical(target, num_classes=10)  
target

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [30]:
from tensorflow.keras import layers, Model

def MobileNetV2(input_shape=(256, 256, 3), num_classes=10):
    inputs = tf.keras.Input(shape=input_shape)
    
    # MobileNetV2 backbone
    backbone = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)
    
    # Add custom head for classification
    x = layers.GlobalAveragePooling2D()(backbone)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs, name='MobileNetV2')
    return model

model = MobileNetV2()
model.summary()



/tmp/ipykernel_103108/1340656753.py:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  backbone = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')(inputs)


Model: "MobileNetV2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 8, 8, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,423,242 (9.24 MB)

 Trainable params: 2,389,130 (9.11 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [32]:
history = model.fit(images, target, epochs=50, verbose=1)
predictions = model.predict(images)
# binary_predictions = (predictions > 0.5).astype(int)
class_predictions = np.argmax(predictions, axis=1)

print(class_predictions)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.2262 - loss: 2.2533
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.2661 - loss: 2.0537
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.2817 - loss: 2.0281
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.2963 - loss: 1.9653
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.2818 - loss: 1.9992
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.3037 - loss: 1.9663
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.3159 - loss: 1.9789
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.3224 - loss: 1.9164
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.3075 - loss: 1.9090
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.3570 - loss: 1.8383
Epoch 11/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.3525 - loss: 1.8191
Epoch 12/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.3294 - lo

KeyboardInterrupt: 

In [22]:
# 모델 저장
model.save('mobilenet_model.h5')  # HDF5 형식으로 저장




In [23]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

# 이미지 디렉토리 경로
image_dir = 'mini_test'  # 경로 수정 필요

# 이미지 읽기 및 전처리
test_images = []
filenames = []
for filename in os.listdir(image_dir):
    if filename.endswith(('.jpg', '.png')):  # 여러 확장자를 처리
        img_path = os.path.join(image_dir, filename)
        try:
            image = Image.open(img_path).convert('RGB').resize((256, 256))  # 이미지 크기 조정 및 RGB 변환
            image_array = img_to_array(image) / 255.0  # 정규화
            test_images.append(image_array)
            filenames.append(filename)  # 파일 이름을 저장하여 나중에 매칭
        except Exception as e:
            print(f"Error loading image {filename}: {e}")

# 이미지 데이터를 NumPy 배열로 변환
test_images = np.array(test_images)

# 예측 수행
prediction = model.predict(test_images)

# 클래스 예측값을 가져옴 (argmax로 각 이미지에 대해 가장 높은 확률의 클래스 선택)
class_predictions = np.argmax(prediction, axis=1)

# 이미지 파일 이름과 예측 결과를 매칭하여 출력
for i, filename in enumerate(filenames):
    print(f"Image: {filename} - Predicted Class: {class_predictions[i]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Image: 1. 저인망 조업.jpg - Predicted Class: 8
Image: 1. 타망 조업.jpg - Predicted Class: 8
Image: 1.등광조망 조업.jpg - Predicted Class: 8
Image: 11. 유망 조업.jpg - Predicted Class: 8
Image: 23. 등광조망 조업.jpg - Predicted Class: 8
Image: 5. 등광조망 조업.jpg - Predicted Class: 8
Image: 5. 저인망 조업.jpg - Predicted Class: 8
Image: 5. 타망 조업.jpg - Predicted Class: 8
Image: 6. 낚시어선 이동.jpg - Predicted Class: 8
Image: 7. 유망 조업.jpg - Predicted Class: 8
Image: 9. 등광조망 조업.jpg - Predicted Class: 8
Image: 9. 유망 조업.jpg - Predicted Class: 8
Image: 9. 저인망 조업.jpg - Predicted Class: 8
Image: 9. 타망 조업.jpg - Predicted Class: 8


타망: 0
유망: 1
등광조망: 2
범장망: 3
안강망: 4
연승: 5
통발: 6
저인망: 7
낚시어선: 8
채낚기: 9